In [0]:

from aadatapipelinecore.core.urn import Urn
import psycopg2
spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
from aaplproxy.connection import ClusterConnection
from conf import settings
from aadatapipelinecore.core.loader.plproxy import build_db_settings
import datetime as d
import datetime
from datetime import timedelta
PG_AA_HOSTS = [('10.2.26.136', 6432)]
PG_AA_NAME = 'aa'
PG_AA_ACCESS_ID = 'app_bdp_usage_qa'
PG_AA_SECRET_KEY = '2mHdFW6%#REu'

aa_dsn = (
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db=PG_AA_NAME,
        user=PG_AA_ACCESS_ID,
        host=PG_AA_HOSTS[0][0],
        password=PG_AA_SECRET_KEY,
        port=PG_AA_HOSTS[0][1]
    )
)

urn = Urn(
    namespace="app-qa.db-check.v1",
    owner="app_qa"
)
sql= """select count(distinct name) from aa_domain_metadata;"""
def query(dsn, sql):
    with psycopg2.connect(dsn) as conn:
        conn.autocommit = True
        with conn.cursor() as cur:
            cur.execute(sql)
            result = cur.fetchall()
            conn.commit()
    return result
    
now = d.datetime.now()
today = now.strftime("%Y-%m-%d")
prev_day = (now - timedelta(days=7)).strftime("%Y-%m-%d")
sql = """select count(*) from aa_domain_metadata 
        where DATE(update_time) between '{prev_day}' and '{today}';"""
upt_cnt = [query(aa_dsn, sql.format(prev_day=prev_day, today=today))[0][0]]
        
print upt_cnt

In [0]:
%%sh
# aws s3 rm --recursive s3://b2c-prod-data-pipeline-qa/aa.mobile.web.domain/2020-03-25/
aws s3 ls s3://b2c-prod-data-pipeline-qa/aa.mobile.web.domain/

In [0]:


# spark.read.parquet("s3://b2c-prod-data-pipeline-qa/aa.mobile.web.domain/2020-04-01/").show()
spark.read.parquet("s3://b2c-prod-data-pipeline-qa/aa.mobile.web.domain/2020-03-25/").show()

# 0401 406255
# 0325 405880
# meta_date = ["2020-03-25"]
# meta_count = [1]

# df = spark.createDataFrame(zip(meta_date, meta_count), schema=['date', 'count'])
# df.show()
# df.write.mode('overwrite').parquet("s3://b2c-prod-data-pipeline-qa/aa.mobile.web.domain/2020-03-25/")

In [0]:

from aadatapipelinecore.core.urn import Urn
import psycopg2
spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
from aaplproxy.connection import ClusterConnection
from conf import settings
from aadatapipelinecore.core.loader.plproxy import build_db_settings
import datetime as d
import datetime
from datetime import timedelta
from applications.db_check_v1.common.base_test import PipelineTest

from applications.db_check_v1.cases.store.market_size_v1.test_market_size_v1 import TestMarketSizeWeekly

from applications.db_check_v1.common.utils import string_to_datetime

from applications.db_check_v1.cases.advanced_review.test_advainced_review import TestAdvancedReview
# from applications.db_check_v1.cases.aso.test_aso import TestASO, TestASOSOVDataDaily, \
#     TestASOSOVDataWeekly, TestASOSOVDataMonthly, TestASOMetrics
from applications.db_check_v1.cases.market.test_market import TestMarketDimensionDaily, \
    TestMarketLogsFactAndSeenDaily, TestMarketMonthlyCheck, TestMarketWeeklyCheck
from applications.db_check_v1.cases.mobile_web.test_mobile_web import TestMobileWebDaily, \
    TestMobileWebWeekly, \
    TestMobileWebMonthly, TestMobileWebRetention
from applications.db_check_v1.cases.sdk.test_sdk import TestSDKDaily, TestSDKWeekly, TestSDKMonthly
from applications.db_check_v1.cases.store.app_rank_v1.test_app_rank_v1 import TestAppStoreRankDaily
from applications.db_check_v1.cases.store.app_rank_v1.test_app_store_rank import TestAppStoreRank
from applications.db_check_v1.cases.store.download_attribution.test_download_attribution import \
    TestDownloadAttribution
from applications.db_check_v1.cases.store.market_size_v1.test_market_size_v1 import TestMarketSizeWeekly
from applications.db_check_v1.cases.usage.test_city_level import TestCityLevelWeekly, TestCityLevelMonthly
from applications.db_check_v1.common.html_report_test_runner import HTMLTestRunner
#

import sys
import datetime
import traceback
import unittest
PG_AA_HOSTS = [('10.2.26.136', 6432)]
PG_AA_NAME = 'aa'
PG_AA_ACCESS_ID = 'app_bdp_usage_qa'
PG_AA_SECRET_KEY = '2mHdFW6%#REu'

aa_dsn = (
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db=PG_AA_NAME,
        user=PG_AA_ACCESS_ID,
        host=PG_AA_HOSTS[0][0],
        password=PG_AA_SECRET_KEY,
        port=PG_AA_HOSTS[0][1]
    )
)
def query(dsn, sql):
    with psycopg2.connect(dsn) as conn:
        conn.autocommit = True
        with conn.cursor() as cur:
            cur.execute(sql)
            result = cur.fetchall()
            conn.commit()
    return result
class TestMobilWebDomainMeta(PipelineTest):
    trigger_date_config = ("0 12 * * 4", 0)

    def setUp(self):
        super(TestMobilWebDomainMeta, self).setUp()
        self.prev_day = (string_to_datetime(self.check_date, "%Y-%m-%d") - datetime.timedelta(days=7)).strftime(
            "%Y-%m-%d")

    def test_mobile_domain_meta(self):
        # check previous 7 days update.
        failed_results = []
        sql = """select count(*) from aa_domain_metadata where DATE(update_time) between '{prev_day}' and '{today}';"""
        upt_cnt = [query(aa_dsn, sql.format(prev_day=self.prev_day, today=self.check_date))[0][0]]
        if upt_cnt == 0:
            failed_results = [{self.check_date: upt_cnt}]
        self.assertEqual(upt_cnt, 0, "{}".format(upt_cnt))
        self.assertTrue(len(failed_results) == 0, failed_results)
def debug(case_list):
    std_out_origin= sys.stdout
    std_err_origin= sys.stderr
    try:
        suite =  unittest.TestSuite()
        for case in case_list:
            suite.addTest(case)
        runner = unittest.TextTestRunner(verbosity=2, buffer=True)
        runner.run(suite)
    except Exception as ex:
        print dir(ex)
        print ex.message
        traceback.print_exception(type(ex), ex, ex.__traceback__)
    finally:
        sys.stdout = std_out_origin
        sys.stderr = std_err_origin
    
case_name_list = [
    "test_mobile_domain_meta",
    
    ]

# TestMarketDimensionDaily(trriger_date='2020-09-01')
test_case_list = [ ] # pass 
test_case_list.append(unittest.TestLoader().loadTestsFromTestCase(TestMobilWebDomainMeta))
# test_case_list.append(unittest.TestLoader().loadTestsFromTestCase(TestMarketLogsFactAndSeenDaily))
# test_case_list.append(unittest.TestLoader().loadTestsFromTestCase(TestMarketWeeklyCheck))
# test_case_list.append(unittest.TestLoader().loadTestsFromTestCase(TestMarketMonthlyCheck))


debug(test_case_list)


In [0]:
%%sh


PGPASSWORD='2mHdFW6%#REu' psql -h 10.2.26.136  -U app_bdp_usage_qa -d dna -p 6432 << EOF 
\c aa
select count(*) from aa_domain_metadata where DATE(update_time) between '2020-03-19' and '2020-03-26';

EOF

In [0]:

trigger_date_config = ("0 12 * * 4", 0)
check_date = _get_date_from_refresh_routing_config(trigger_date_config)
print check_date